This [notebook](https://atmos.washington.edu/~nbren12/reports/uwnet/notebooks/2019-05-14-water-budget-first-2-days-MESSY.html) showed that the net precipitation predictions in days 0-2 are pretty good, but the vertical velocity is still too small. **Is this occuring because the net heating is too small?** Overall, we have focused a lot on the moisture budget, but the heating is what creates buoyancy and vertical motions. 

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import torch
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
import numpy as np
import xarray as xr

from src.data import SAMRun, NGAqua
from uwnet import thermo

def plot_tropics_mean_pw(case, key):
    z= case.data_2d[key].sel(y=slice(4.5e6, 5.5e6), time=slice(None, 110)).mean(['x', 'y'])
    z.plot()

    
def plot_tropics_avg_cases(runs, key='PW'):
    labels = []
    for case, label in runs:
        plot_tropics_mean_pw(case, key)
        labels.append(label)
    plt.legend(labels)
    plt.title(f"Tropical average of {key}")

    

In [ ]:
ng = NGAqua("../data/raw/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/")

low_res = "../data/runs/2019-05-14-model268-epoch5-debiased-res128x64x34/"
lo_run = SAMRun(low_res, case='control')

# this is expensive. so preloads
data_2d = ng.data_2d

# compute water budgets
ng_h20 = thermo.water_budget(data_2d.sel(time=slice(100, 110)))
lo_h20 = thermo.water_budget(lo_run.data_2d.sel(time=slice(100, 110)))

# runs
runs = [(ng, 'NG-Aqua'), (lo_run, 'NN')]

In [ ]:
plot_tropics_avg_cases(runs)

In [ ]:
plot_tropics_avg_cases(runs, key='NPNN')

In [ ]:
plot_tropics_avg_cases(runs, key='NHNN')

In [ ]:
plot_tropics_avg_cases(runs, key='W500')

These results show that the net heating (NHNN) is accuratei n the first day, although the peak is understimated during day time. Similarly, W500 does not have a strong diurnal cycle, even though net heating (NHNN) does.

In [ ]:
net_heating_nn = lo_run.data_2d.NHNN
net_heating_ng = ng.data_2d.NHNN.interp_like(net_heating_nn)

In [ ]:
net_heating_ng.mean('x').plot()

In [ ]:
net_heating_nn.mean('x').plot()